# Hyperparameter search 

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras import layers
#import keras

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

import hvplot.pandas
import panel as pn
import holoviews as hv
hv.extension('bokeh')

import os

In [ ]:
dflist = [pd.read_excel('./dsm2_ann_BaselineData_20220120.xlsx',i,index_col=0,parse_dates=True) for i in range(8)]

In [ ]:
dfinps = pd.concat(dflist[0:7],axis=1)
dfinps.head()

In [ ]:
dfouts = dflist[7]

In [ ]:
dfouts

## Tensorflow Board Setup
A log directory to keep the training logs

Tensorboard starts a separate process and is best started from the command line. Open a command window and activate this environment (i.e. keras) and goto the current directory. Then type in
```
tensorboard --logdir=./tf_training_logs/ --port=6006
```

In [ ]:
from tensorflow import keras

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir=./tf_training_logs/ --port=6006
#root_logdir = os.path.join(os.curdir, "tf_training_logs")
#tensorboard_cb = keras.callbacks.TensorBoard(root_logdir)## Tensorflow Board Setup

In [ ]:
output_locations = list(dfouts.columns)
calib_slice = slice('2000', '2020')
valid_slice = slice('1990', '2000')

In [ ]:
import keras_tuner as kt

In [ ]:
# Define Sequential model with 3 layers
NFEATURES = 126  # (8 + 10)*7


def build_model(hp):
    #nhidden1=8, nhidden2=2, act_func='sigmoid'
    act_func=hp.Choice("activation", ["relu", "tanh","elu","sigmoid"])
    model = keras.Sequential(
        [
            layers.Input(shape=(NFEATURES)),
            #layers.Flatten(),
            layers.Dense(hp.Int("nhidden1", min_value=2, max_value=32, step=4), activation=act_func),
            layers.Dense(hp.Int("nhidden2", min_value=2, max_value=16, step=2), activation=act_func),
            layers.Dense(1, activation=keras.activations.linear)
        ])
    model.compile(optimizer=keras.optimizers.Adam(
        learning_rate=0.001), loss="mse")
    #model.compile(optimizer=keras.optimizers.RMSprop(), loss="mse")
    return model

In [ ]:
build_model(kt.HyperParameters())

In [ ]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_loss",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory='tuner',
    project_name="ann_dsm2",
)

In [ ]:
tuner.search_space_summary()

In [ ]:
import annutils

In [ ]:
location = 'Antioch'
output_location = '%s' % location
# create tuple of calibration and validation sets and the xscaler and yscaler on the combined inputs
(xallc, yallc), (xallv, yallv), xscaler, yscaler = \
    annutils.create_training_sets([dfinps],
                                  [dfouts[[output_location]]],
                                  calib_slice=calib_slice,
                                  valid_slice=valid_slice)
tuner.search(xallc, yallc,
             epochs=1000,
             batch_size=128,
             validation_data=(xallv, yallv),
             callbacks=[
                 keras.callbacks.EarlyStopping(
                     monitor="val_loss", patience=50, mode="min", restore_best_weights=True),
                 #tensorboard_cb
             ],
             )